In [4]:
import pandas as pd
from collections import Counter
import string
import jctconv
import emoji
import re

In [9]:
df = pd.read_csv('all_rakuma.csv')
df.head(2)

,date,item_url,title,price,brand,category,good,bad,normal,ship_days,comments,last_log_date,good_counts,prefecture,delivery_fee,seller_url,size,picture_url,description,details_dict
0,2019/10/08 23:39:51,https://item.fril.jp/055f4080a2a0ae275db6ba7be...,Epiphone LTD Korina Explorer Bass,40000,Epiphone,楽器 > ベース > エレキベース,109,0,0,1-2日後,NaN,2000-01-01,0,岐阜県,着払い,https://fril.jp/shop/42df208014145c438f96e01f5...,なし,https://img.fril.jp/img/259586636/l/734809353....,コリーナボディ ブラックピックガード ゴールドパーツの豪華なエクスプローラー コリーナ ...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', 'ブランド':..."
1,2019/10/08 23:39:51,https://item.fril.jp/896a1d0197d05cf8431c136d9...,TVB-100 (SUN),38000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,"['TOKYO GUITARS', '本モデルを含め、TOKYO GUITARS工房にて製造...",2019/08/31,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/223532100/l/635334018....,Tokyo Guitars モデル名 TVB 100 SUN Tokyo Cust...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."


In [13]:
ctg_df = df.category.value_counts().reset_index()

In [16]:
over_100data = ctg_df[ctg_df.category > 1000]

In [17]:
over_100data.category.sum()

63006

In [19]:
data = df.merge(over_100data[['index']], left_on='category', right_on='index')

In [21]:
data.category.nunique()

22

In [22]:
data.title

0           テレカ 未使用品 仮面ライダー 生誕20周年 東映ビデオ販売 販促用非売品
1           送料無料 新品 DVD Perfume WORLD TOUR 1st 初回
2                          ☆ウルトラマン☆レーザーディスク ジャンク？
3             MAISHA （マイシャ）Sadao Watanabe　渡辺貞夫★LD
4        LD★PARKER'S MOOD（パーカーズ・ムード）LIVE AT BRAVA
                           ...                   
63001                                   64 マリオテニス
63002    ☆非売品☆switch ポケットモンスター ポケモンカードバトル クリアファイル
63003    【新品未開封】大乱闘スマッシュブラザーズ SPECIAL（クロスクリーナー付き）
63004    【新品未開封】大乱闘スマッシュブラザーズ SPECIAL（クロスクリーナー付き）
63005    【新品未開封】大乱闘スマッシュブラザーズ SPECIAL（クロスクリーナー付き）
Name: title, Length: 63006, dtype: object

In [39]:
emojis = ''.join(emoji.UNICODE_EMOJI.keys())

In [40]:
puncs = string.punctuation + "◆▼★②●☆■★【】『』「」、♪"

In [41]:
def han2zen(txt):
    txt = jctconv.h2z(txt, kana=True, digit=False, ascii=False)
    return jctconv.z2h(txt, kana=False, digit=True, ascii=True)

def remove_signs(txt):
    rm_signs = emojis + puncs
    for s in rm_signs:
        txt = txt.replace(s, ' ')
    return txt

def clean_txt(txt):
    txt = han2zen(txt)
    txt = remove_signs(txt)
    txt_list = txt.upper().split()
    txt_list = [x for x in txt_list if len(x) > 1 and re.search(r'[亜-熙ぁ-んァ-ヶa-zA-Z]', x)]
    return list(Counter(txt_list))

In [67]:
data["clean_title"] = data.title.apply(clean_txt)

In [68]:
data.tail().title.apply(clean_txt)

63001                                       [マリオテニス]
63002      [非売品, CH, ポケットモンスター, ポケモンカードバトル, クリアファイル]
63003    [新品未開封, 大乱闘スマッシュブラザーズ, SPECIAL, クロスクリーナー付き]
63004    [新品未開封, 大乱闘スマッシュブラザーズ, SPECIAL, クロスクリーナー付き]
63005    [新品未開封, 大乱闘スマッシュブラザーズ, SPECIAL, クロスクリーナー付き]
Name: title, dtype: object

In [69]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB


In [70]:
category2idx = {c: idx for idx, c in enumerate(data.category.unique())}
idx2category = {idx: c for idx, c in enumerate(data.category.unique())}

In [71]:
X = data.title
y = data.category.apply(lambda x: category2idx[x])

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [77]:
pipeline = Pipeline([
    ('bow', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('classifier', RandomForestClassifier()),
])

In [78]:
pipeline.fit(X_train, y_train)


Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', RandomForestClassifier())])

In [79]:
pred = pipeline.predict(X_test)

In [80]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.87      0.64      0.74       406
           1       0.90      0.93      0.91      1189
           2       0.87      0.76      0.81       422
           3       0.88      0.82      0.85       699
           4       0.62      0.89      0.73      3577
           5       0.72      0.38      0.49       395
           6       0.89      0.81      0.85      2138
           7       0.85      0.80      0.83      1033
           8       0.94      0.87      0.90       460
           9       0.96      0.82      0.88       379
          10       0.97      0.84      0.90       871
          11       0.80      0.57      0.67       481
          12       0.87      0.74      0.80       430
          13       0.93      0.78      0.85       399
          14       0.99      0.95      0.97       535
          15       0.95      0.90      0.92       830
          16       0.99      0.79      0.88       396
          17       0.99    

In [90]:
target = ['AIR MORE UPTEMPO ‘96 921948-102 26cm']
prediction = pipeline.predict(target)
idx2category[prediction[0]]

'メンズ > 靴/シューズ > スニーカー'

In [91]:
import pickle

In [92]:
with open('rdmf.pickle', mode='wb') as f:
    pickle.dump(pipeline, f)

In [93]:
with open('rdmf.pickle', mode='rb') as ff:
    model = pickle.load(ff)

In [94]:
model.predict(target)

array([17])

In [96]:
pd.DataFrame([{'k': k, 'v': v} for k, v in  idx2category.items()]).to_csv('idx2category.csv', index=False)

In [97]:
ls

all_rakuma.csv    rakuma_NLP.ipynb  test.py
idx2category.csv  rdmf.pickle       venv/
